In [1]:
!pip install tensorflow gym keras-rl2 gym[atari]

In [2]:
import gym
import random

In [3]:
env = gym.make('MsPacman-v0')#, render_mode='human')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [4]:
env.unwrapped.get_action_meanings()

['NOOP',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT']

In [5]:
episodes = 5
run = False
if run:
  for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0

    while not done:
      env.render()
      action = random.choice([0,1,2,3,4,5])
      n_state, reward, done, info = env.step(action)
      score += reward
    print('Episode:{} Score:{}'.format(episode, score))
  env.close

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam, SGD

In [7]:
def build_model(height, width, channels, actions):
    model = Sequential(layers=[
        Convolution2D(32, (8, 8), strides = (4,4), activation='relu', input_shape = (3,height,width,channels)),
        Convolution2D(64, (4, 4), strides = (2,2), activation='relu'),
        Convolution2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(actions, activation='linear')])
    print(model.summary())
    return model


In [8]:
model = build_model(height, width, channels, actions)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten (Flatten)           (None, 67584)             0         
                                                                 
 dense (Dense)               (None, 512)               34603520  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 9)                 2

In [9]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [10]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(
        model=model, 
        memory=memory, 
        policy=policy, 
        enable_dueling_network=True, 
        dueling_type='avg', 
        nb_actions=actions,
        nb_steps_warmup=1000
    )
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)